# 一等函数

## 5.1 把函数视作对象

In [1]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'returns n!'

In [4]:
type(factorial)

function

In [6]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## 5.2 高阶函数

In [7]:
list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [8]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

## 5.3 匿名函数

In [9]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## 5.4 可调用对象

In [10]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

## 5.5 用户定义的可调用类型

In [12]:
import random

class BingoCage:
    def __init__(self, items):
        self.__items = list(items)
        random.shuffle(self.__items)
    
    def pick(self):
        try:
            return self.__items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()

In [14]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [15]:
bingo()

1

In [16]:
callable(bingo)

True

## 函数内省

In [17]:
dir(factorial)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## 5.7 从定位参数到仅限关键字参数

In [20]:
def tag(name, *content, cls=None, **attrs):
    '''generate one or more HTML tags'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value) for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [21]:
tag('br')

'<br />'

In [22]:
tag('p', 'hello')

'<p>hello</p>'

In [24]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


In [25]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [26]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [29]:
tag(content='testing', name='img')

'<img content="testing" />'

In [30]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [31]:
# if you don't want to specify any parameters, but still want to use keyword-only arguments, put a * by itself in the signature
def f(a, *, b): # attention: the default value of b is not necessarily required
    return a, b

In [32]:
f(1, b=2)

(1, 2)

## 5.8 获取关于参数的信息

In [33]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [34]:
clip.__defaults__

(80,)

In [35]:
clip.__code__

<code object clip at 0x000001E06FE9FE10, file "C:\Users\KOBAYASHI\AppData\Local\Temp\ipykernel_24096\4145452495.py", line 1>

In [36]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [38]:
clip.__code__.co_argcount

2

In [39]:
from inspect import signature
sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [40]:
str(sig)

'(text, max_len=80)'

In [41]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


## 5.9 函数注解

In [43]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """Return text clipped at the last space before or after max_len"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [44]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

In [45]:
from inspect import signature
sig = signature(clip)
sig.return_annotation

str

In [46]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


## 5.10 支持函数式编程的包

In [47]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

In [48]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
]

In [50]:
from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(2)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))


In [53]:
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [59]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) for name, cc, pop, (lat, long) in metro_data]

In [60]:
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [61]:
metro_areas[0].coord.lat

35.689722

In [62]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')

In [63]:
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [64]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [65]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [66]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

In [67]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
triple(7)

21

In [68]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [69]:
import unicodedata, functools
nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [70]:
s1 == s2

False

In [71]:
nfc(s1) == nfc(s2)

True

In [72]:
tag

<function __main__.tag(name, *content, cls=None, **attrs)>

In [73]:
from functools import partial
picture = partial(tag, 'img', cls='pic-frame')
picture(src='wumpus.jpeg')

'<img class="pic-frame" src="wumpus.jpeg" />'